### Importa as bibliotecas a serem usadas

In [74]:
from pathlib import Path
import pandas as pd
import numpy as np
import xarray as xr
import math

### Define caminho onde os dados .VEC estao salvos e cria uma pasta Dados_csv, onde amarzenara os dados gerados pelo script

In [2]:
caminho = Path('dados/tidi/')
arqs = sorted([str(i) for i in caminho.glob('**/*.VEC')])
arquivos_csv = Path('dados/tidi/dados_csv')
arquivos_csv.mkdir(exist_ok=True)

### Loop para cada arquivo .VEC
### df.to_csv salva os dados

In [3]:
for arquivo in arqs:
    # comando para ler arquivo no formato netcdf
    df=xr.open_dataset(arquivo, engine='scipy')
    df= df.to_dataframe()
    #selecionando variaveis
    df2=df[['ascending','alt_retrieved','ut_date','ut_time','lat','lon','data_ok','u','var_u','v','var_v']]
    #selecionando latitude de São João do Cariri
    df2=df2.loc[df['lat'] >= -17]
    df2=df2.loc[df['lat'] <= 3]
    #selecionando longitude de São João do Cariri
    df2=df2.loc[df['lon'] >= 314]
    df2=df2.loc[df['lon'] <= 334]
    #Selecionando o intervalo de altitude
    df2=df2.loc[df['alt_retrieved'] >= 70]
    df2=df2.loc[df['alt_retrieved'] <= 100]
    df.ut_time
    time2=df.ut_time
    #convertendo o tempo para unidade de hora
    time2=time2/1e3/6e1/6e1
    df2['UT_time']=time2

    df2=df2[['ascending','alt_retrieved','ut_date','UT_time','lat','lon','data_ok','u','var_u','v','var_v']]
    df2
    # comando para salvar os arquivos
    #your_name= nome para salvar os arquivos
    df2.to_csv(f'dados/tidi/dados_csv/{str(arquivo)[16:-4]}.csv', sep=';')

### Converter dados csv para txt com ventos zonal e meridional

<strong> u = Zonal and v = Meridional

<strong> arquivo final: zonal/meridional_yyyymmdd <br>
<strong> colunas do arquivo final: tempo_corrido, 70km ... 100km, std_70km ... std_100km 

In [2]:
dict_months = {
    '01': [1, 31],
    '02': [32, 59],
    '03': [60, 90],
    '04': [91, 120],
    '05': [121, 151],
    '06': [152, 181],
    '07': [182, 212],
    '08': [213, 243],
    '09': [244, 273],
    '10': [274, 304],
    '11': [305, 334],
    '12': [335, 365],
}

In [ ]:
dict_day_to_month = { for for k,v in zip(dict_months.keys(), dict_months.values())}

In [85]:
for k,v in zip(dict_months.keys(), dict_months.values()):
    print(v)

[1, 31]
[32, 59]
[60, 90]
[91, 120]
[121, 151]
[152, 181]
[182, 212]
[213, 243]
[244, 273]
[274, 304]
[305, 334]
[335, 365]


In [62]:
df = pd.read_csv('dados/tidi/dados_csv/TIDI_PB_2005001_P0100_S0200_D007_R01.csv', sep=';')

In [63]:
lista_tempos = list(df.UT_time.value_counts().index)
lista_alt = list(df.alt_retrieved.value_counts().index)
lista_alt_df = [str(i)+'km' for i in lista_alt]
lista_alt_std_df = ['std_'+str(i)+'km' for i in lista_alt]

In [64]:
df_converted = pd.DataFrame(columns=['tempo_decorrido']+lista_alt_df+lista_alt_std_df, index=range(0, len(lista_tempos)))

In [65]:
df_converted

,tempo_decorrido,70.0km,72.5km,75.0km,77.5km,80.0km,82.5km,85.0km,87.5km,90.0km,...,std_77.5km,std_80.0km,std_82.5km,std_85.0km,std_87.5km,std_90.0km,std_92.5km,std_95.0km,std_97.5km,std_100.0km
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
df_ = df[df.UT_time==lista_tempos[0]][['UT_time', 'alt_retrieved', 'u', 'var_u']].T

In [67]:
df_

,0,8,16,24,32,40,48,56,64,72,80,88,96
UT_time,4.237222,4.237222,4.237222,4.237222,4.237222,4.237222,4.237222,4.237222,4.237222,4.237222,4.237222,4.237222,4.237222
alt_retrieved,70.000000,72.500000,75.000000,77.500000,80.000000,82.500000,85.000000,87.500000,90.000000,92.500000,95.000000,97.500000,100.000000
u,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-44.368134,-108.823490,-76.288770,-15.324519,13.917387,31.711498
var_u,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.668412,0.546056,0.546023,0.546016,0.546034,0.546064


In [68]:
df_converted.iloc[0]['tempo_decorrido'] = df_[0]['UT_time']

In [61]:
df_[0]['u']

nan

In [78]:
for tempo,i in zip(lista_tempos, range(len(lista_tempos))):
    df_ = df[df.UT_time==tempo][['UT_time', 'alt_retrieved', 'u', 'var_u']].T
    df_converted.iloc[i]['tempo_decorrido'] = df_[i]['UT_time']
    for col, alt_df, alt_std_df in zip(df_.columns, lista_alt_df, lista_alt_std_df):
        df_converted.iloc[i][alt_df] = df_[col]['u']
        df_converted.iloc[i][alt_std_df] = math.sqrt(df_[col]['var_u'])

In [79]:
df_converted.T

,0,1,2,3,4,5,6,7
tempo_decorrido,4.237222,4.273333,4.309444,7.584722,16.322778,16.358611,17.976944,18.013056
70.0km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72.5km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75.0km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77.5km,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80.0km,NaN,NaN,NaN,NaN,50.234055,-133.11732,-105.99968,-0.593683
82.5km,NaN,NaN,NaN,NaN,31.20158,-103.64053,-66.5027,-2.626763
85.0km,NaN,NaN,NaN,NaN,11.91256,-66.03551,-41.549202,-14.726082
87.5km,-44.368134,-202.62608,NaN,NaN,-0.506615,-29.577728,-3.627695,-22.35768
90.0km,-108.82349,-157.01917,NaN,NaN,-2.041718,-21.928432,23.411177,-20.892956


In [ ]:
caminho = Path('dados/tidi/dados_csv/')
arquivos = sorted([i for i in caminho.glob('*.csv')])